<h2>Simple MNIST convnet - SR Nov21</h2>c.f. https://keras.io/examples/vision/mnist_convnet/

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

2022-12-14 12:22:37.184811: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-14 12:22:37.340820: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-14 12:22:37.353797: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-14 12:22:37.353824: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [2]:
num_classes = 10
input_shape = (28, 28, 1)
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 [==============================] - 18s 2us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:
model = keras.Sequential(
    [   layers.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ])
model.summary()

2022-12-14 12:22:58.062232: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-14 12:22:58.062263: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-14 12:22:58.062288: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (archlinux): /proc/driver/nvidia/version does not exist
2022-12-14 12:22:58.063026: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

In [4]:
batch_size = 128
epochs = 15
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 13s 29ms/step - loss: 0.3570 - accuracy: 0.8905 - val_loss: 0.0812 - val_accuracy: 0.9785
Epoch 2/15
422/422 [==============================] - 12s 29ms/step - loss: 0.1135 - accuracy: 0.9654 - val_loss: 0.0604 - val_accuracy: 0.9828
Epoch 3/15
422/422 [==============================] - 12s 29ms/step - loss: 0.0867 - accuracy: 0.9731 - val_loss: 0.0488 - val_accuracy: 0.9872
Epoch 4/15
422/422 [==============================] - 12s 28ms/step - loss: 0.0730 - accuracy: 0.9772 - val_loss: 0.0410 - val_accuracy: 0.9887
Epoch 5/15
422/422 [==============================] - 6s 15ms/step - loss: 0.0647 - accuracy: 0.9797 - val_loss: 0.0390 - val_accuracy: 0.9893
Epoch 6/15
422/422 [==============================] - 6s 14ms/step - loss: 0.0583 - accuracy: 0.9818 - val_loss: 0.0348 - val_accuracy: 0.9908
Epoch 7/15
422/422 [==============================] - 6s 14ms/step - loss: 0.0541 - accuracy: 0.9832 - val_loss: 0.0364 - val_accuracy: 0.

In [5]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.024792244657874107
Test accuracy: 0.9919999837875366


OK, now let us play with the network and supply it with our own data...

In [6]:
x_test.shape, y_test.shape

((10000, 28, 28, 1), (10000, 10))

In [7]:
model.predict(x_test[0].reshape(1,28,28,1))

1/1 [==============================] - 0s 45ms/step


array([[1.3655592e-10, 1.4007576e-11, 8.3530074e-07, 1.1018266e-06,
        5.2573668e-14, 1.1259962e-10, 8.9695247e-18, 9.9999797e-01,
        1.2843427e-09, 6.6535641e-08]], dtype=float32)

In [8]:
test=1-np.fromfile('test-image2.data',dtype=np.ubyte)/255
model.predict(x_test[0].reshape(1,28,28,1))

FileNotFoundError: [Errno 2] No such file or directory: 'test-image2.data'